In [44]:
# import QGRID para mejor visualizacion de los datos
# los pptos sin abonos se deben cancelar en ClinicCloud, cosa q no generen alertas. O q el paciente tenga saldo a favor
# Debemos Generar un archivo identificandolos. Las ff´s deben tener algun ingreso de dinero para no cancelarlas.
# Los pptos aceptados hoy pueden haber sido elaborados en una fecha anterior. Su fecha se debe actualizar a la de hoy, 
# ya sea en ClinicCloud o en el reporte (mejor - 'fecha_inicio')

import numpy as np
import pandas as pd
import time

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
pd.set_option('display.max_colwidth', -1)

In [45]:
archivo_PPE = 'Produccion_pendiente_exigible_con_tratamientos_Dentix_Calle_90.csv'
archivo_crono = 'crono_Dentix_Calle_90_1.csv'

In [46]:
crono_df = pd.read_csv(archivo_crono, encoding = 'unicode_escape', sep=',')
PPE = pd.read_csv(archivo_PPE, encoding = 'unicode_escape', sep=',')

#Eliminación de Puntos y Revisiones del PPE
PPE.set_index('Servicio', inplace=True)
PPE.drop(index='Puntos y Revisiones.  ',inplace=True)
PPE.reset_index(inplace=True)
PPE['duplicated'] = PPE.duplicated()
PPE.set_index('duplicated',inplace=True)
PPE.drop(True,inplace=True)
PPE.reset_index(inplace=True)
PPE.drop(columns=['duplicated'],inplace=True)
PPE

,Servicio,Clínica,Fecha Pres.,Nº Historia,Presupuesto,Descuento,Teléfono Paciente,Doctor,Especialidad,Explicado,Importe Pres.,Pagado Pres.,% Pagado,% Realizado,Producidos,Pendientes Producción,Próxima Cita,Fecha Última Cita Prod
0,RESINA FOTOPOLIMERIZADO 1 SUP POSTERIOR Y ANTERIOR. Pieza: 35,Dentix Calle 90,01/09/2018,BG057272,Nº 4159 - DIANA CAROLINA ZUZUNAGA RUIZ DE MONTES,0,3207331238,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,CLAUDIA LORENA TAMAYO JIMENEZ,79000,0,0.00%,0.00%,0,79000,07/09/2018,Sin Cita Producida
1,DETARTRAJE POR ARCADA.,Dentix Calle 90,01/09/2018,BG056739,Nº 4160 - ANDRIA BLAGOJEVIC,0,3204880245,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,97520,0,0.00%,0.00%,0,97500,07/09/2018,Sin Cita Producida
2,PROFILAXIS. Pieza: 11,Dentix Calle 90,01/09/2018,BG056739,Nº 4160 - ANDRIA BLAGOJEVIC,0,3204880245,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,40000,0,0.00%,0.00%,0,40000,07/09/2018,Sin Cita Producida
3,DETARTRAJE POR ARCADA.,Dentix Calle 90,01/09/2018,BG055264,Nº 4161 - MICHELLE KRISTINE WARREN,0,3204880245,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,97520,0,0.00%,0.00%,0,97500,07/09/2018,Sin Cita Producida
4,PROFILAXIS. Pieza: 11,Dentix Calle 90,01/09/2018,BG055264,Nº 4161 - MICHELLE KRISTINE WARREN,0,3204880245,LAURA ALEJANDRA RONCANCIO GARCIA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,40000,0,0.00%,0.00%,0,40000,07/09/2018,Sin Cita Producida
5,PROFILAXIS. Pieza: 11,Dentix Calle 90,31/08/2018,BG058081,Nº 4139 - camilo armando paez penagos,0,3194585849,MARIA FERNANDA SUAREZ BECERRA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,40000,40000,100%,0.00%,0,40000,No Citado,Sin Cita Producida
6,DETARTRAJE POR ARCADA.,Dentix Calle 90,31/08/2018,BG058081,Nº 4139 - camilo armando paez penagos,0,3194585849,MARIA FERNANDA SUAREZ BECERRA,OPERATORIA GENERAL,SANDRA PATRICIA GONZALEZ TOVAR,97520,97500,100%,0.00%,0,97500,No Citado,Sin Cita Producida
7,EXODONCIA SIMPLE COMPLEJIDAD 1-2-3. Pieza: 63,Dentix Calle 90,31/08/2018,BG058081,Nº 4139 - camilo armando paez penagos,0,3194585849,MARIA FERNANDA SUAREZ BECERRA,CIRUJIA ORAL Y MAXILOFACIAL,SANDRA PATRICIA GONZALEZ TOVAR,93500,93500,100%,0.00%,0,93500,No Citado,Sin Cita Producida
8,EXODONCIA QUIRURGICA DE CORDAL (INCLUIDO SEMIERUPCIONADO). Pieza: 18,Dentix Calle 90,31/08/2018,BG058081,Nº 4139 - camilo armando paez penagos,0,3194585849,MARIA FERNANDA SUAREZ BECERRA,CIRUJIA ORAL Y MAXILOFACIAL,SANDRA PATRICIA GONZALEZ TOVAR,296800,296800,100%,0.00%,0,296800,No Citado,Sin Cita Producida
9,EXODONCIA QUIRURGICA DE CORDAL (INCLUIDO SEMIERUPCIONADO). Pieza: 28,Dentix Calle 90,31/08/2018,BG058081,Nº 4139 - camilo armando paez penagos,0,3194585849,MARIA FERNANDA SUAREZ BECERRA,CIRUJIA ORAL Y MAXILOFACIAL,SANDRA PATRICIA GONZALEZ TOVAR,296800,296800,100%,0.00%,0,296800,No Citado,Sin Cita Producida


In [47]:
print('REGISTROS CRONOGRAMA:',crono_df.count()[0])
print('REGISTROS PPE:',PPE.count()[0])

REGISTROS CRONOGRAMA: 182
REGISTROS PPE: 181


In [48]:
#CRUCE PPE - CRONO: en este punto revisar que cada ttmto del PPE tenga su cronograma. Sinó, sacar los pptos q faltan por crono.

pptos_faltantes = pd.merge(PPE,crono_df,how='left',left_on=['Clínica','Nº Historia','Presupuesto','Servicio'], right_on=['Clínica','Nº Historia','Presupuesto','Servicio'])
pptos_faltantes['Semana'].fillna(0,inplace=True)
pptos_faltantes.set_index('Semana',inplace=True)
pptos_faltantes = pptos_faltantes.loc[0]

pptos_faltantes

KeyError: 'the label [0] is not in the [index]'

In [ ]:
pptos_faltantes.reset_index(inplace=True)
pptos_faltantes['Cantidad Presupuestada'] = (pptos_faltantes['Producidos']+pptos_faltantes['Pendientes Producción'])/pptos_faltantes['Importe Pres.']
pptos_faltantes['Cantidad Presupuestada'] = np.ceil(pptos_faltantes['Cantidad Presupuestada'])
pptos_faltantes['Cantidad Presupuestada'].fillna(1,inplace=True) #las CSI traen vacío el campo 'Pendientes Produccion'.
# en realidad parece que las unicas cantidades presupuestadas q son necesarias son las de Mensualidades de ortodoncia.
# Para CSI, el %Realizado es el que nos sirve para calcular Cantidad Producida.
# la Cantidad Pendiente se puede utilizar para saber la fecha de colocación de un implante si hay ortodoncia.

pptos_faltantes['ttmto_homol'] = pptos_faltantes['Servicio'].str.slice(0,-13)
pptos_faltantes['ttmto_homol'] = pptos_faltantes['ttmto_homol'].str.strip()
pptos_faltantes['pieza'] = pptos_faltantes['Servicio'].str.slice(-2)
pptos_faltantes['pieza'] = pptos_faltantes['pieza'].str.strip()
pptos_faltantes['fecha_inicio'] = time.strftime("%d/%m/%Y")

pptos_faltantes = pptos_faltantes[['Clínica', 'Fecha Pres._x', 'Nº Historia', 'Presupuesto', 
                                   'Servicio', 'Semana', 'Cantidad Presupuestada','Doctor_x', 
                                   'Explicado_x', 'ttmto_homol','pieza', 'fecha_inicio']]

print('Numero de tratamientos faltantes por crono:',pptos_faltantes.count()[0])
pptos_faltantes.to_csv('pptos_faltantes.csv',index = False)

pptos_faltantes.set_index(['Clínica','Nº Historia','Servicio'], inplace=True)
pptos_faltantes